Per visualizzare i grafici, eseguire da menu "Run → Run All Cells"

# Trave isostatica con carico distribuito

Lo scopo di questo esercizio è di valutare l'effetto della posizione degli appoggi sui diagrammi di taglio e momento per una trave sottoposta ad un carico distribuito costante $p$.

In [ ]:
import holoviews as hv
import numpy as np

hv.extension("bokeh")
print(hv.__name__, hv.__version__)
print(np.__name__, np.__version__)

In [ ]:
# holoviews settings and global config
pcolor = hv.Cycle(["red", "orange"])
dt = hv.Dimension("T", range=(-0.5, 0.5), label="T/(pL)")
dm = hv.Dimension("M", range=(-1 / 8, 1 / 8), label="M/(pL^2)")
dx = hv.Dimension("x", range=(0, 1), label="x/L")

%opts Area [height=200 width=600 show_grid=True] (line_width=1.5 fill_color='cornflowerblue')
%opts Points (color=pcolor, size=5)
%opts Curve [height=150 width=600, show_grid=True, tools=["crosshair"]] (color=pcolor)
%opts Layout (tools=["crosshair"])

In [ ]:
def tg(a):
    """crea il diagramma del taglio in funzione della posizione a dell'appoggio"""
    assert 0 <= a <= 0.5
    x1 = np.linspace(start=0, stop=a, endpoint=True)
    t1 = -x1
    m1 = x1**2 / 2
    x2 = np.linspace(start=0, stop=0.5 - a, endpoint=True)
    t2 = t1[-1] + 0.5 - x2
    m2 = m1[-1] - (t1[-1] + 0.5) * x2 + x2**2 / 2
    return (
        hv.Area((x1, -t1), dx, dt)
        * hv.Area((a + x2, -t2))
        * hv.Area((1 - a - x2, t2))
        * hv.Area((1 - x1, t1))
    )

In [ ]:
def mf(a):
    """crea il diagramma del momento flettente in funzione della posizione a dell'appoggio"""
    assert 0 <= a <= 0.5
    x1 = np.linspace(start=0, stop=a, endpoint=True)
    t1 = -x1
    m1 = x1**2 / 2
    x2 = np.linspace(start=0, stop=0.5 - a, endpoint=True)
    t2 = t1[-1] + 0.5 - x2
    m2 = m1[-1] - t2[0] * x2 + x2**2 / 2
    return (
        hv.Area((x1, m1), dx, dm)
        * hv.Area((a + x2, m2))
        * hv.Area((1 - a - x2, m2))
        * hv.Area((1 - x1, m1))
        * hv.Points((a, a**2 / 2))  # .opts(color="red", size=5)
        * hv.Points((0.5, (a - 1 / 4) / 2))  # .opts(color="red", size=5)
    )

In [ ]:
# diagramma del momento flettente in A e B in funzione della posizioen degli appoggi


def pta(x):
    return x**2 / 2


def ptc(x):
    return (x - 1 / 4) / 2


def diag(a):
    x = np.linspace(0, 1 / 2)
    return (
        hv.Curve((x, pta(x)), dx, dm)
        * hv.Curve((x, ptc(x)), dx, dm)
        * hv.Points((a, pta(a)))
        * hv.Points((a, ptc(a)))
    )


def diag_abs(a):
    x = np.linspace(0, 1 / 2, 400)
    return (
        hv.Curve((x, pta(x)), dx, dm)
        * hv.Curve((x, abs(ptc(x))), dx, dm)
        * hv.Points((a, pta(a)))
        * hv.Points((a, abs(ptc(a))))
    ).redim.range(M=(0, 1 / 8))

In [ ]:
hv.DynamicMap(lambda a: (tg(a) + mf(a) + diag(a)).cols(1), kdims=["a"]).redim.values(
    a=np.linspace(0, 0.5, 201)
)

In [ ]:
%%opts Curve [height=300 width=600, show_grid=True]
hv.DynamicMap(
    lambda a: (tg(a) + mf(a) + diag_abs(a)).cols(1), kdims=["a"]
).redim.values(a=np.linspace(0, 0.5, 201))

La soluzione ottimale è
\begin{align}
\frac{a}{L} &= \frac{\sqrt2 - 1}{2} \approx 0.2071 \dots \\
\frac{\lvert M_\max \rvert}{\frac 18 pL^2} &= {3 - 2\sqrt{2}} \approx 0.17157\dots
\end{align}